# 使い方

- ①、②、③をShift+Enterで実行しておく。定義部分。
- ④が実行部分。

```python

product_b_list = b_list(1157,100)#[1,101,201,...,1001,1101]の、b=パラメータに入れる数値を作成。
products = ["airpods","第2世代","新品"] #ここに検索ワードを"リストで"入れる

for number in product_b_list:
    print(str(number)+"商品から取得")
    # b=に1や101を代入。productsにワードのリストを代入。
    # 検索結果一ページごとのリンクタグが100個帰ってくるのでtagsに代入 
    tags = get_search_result_page_single(b=number,products=products)
          
    create_dataframe(detail_link_tags=tags, b=number,query=get_combined_params(products))
    print(str(b+99)+"商品まで取得完了")
```
- 検索ワードを繋げたフォルダの下に ("airpods+第2世代+新品/")
- 検索ワードと番号を繋げた名前のCSVが100件ごとに入っているので、("airpods+第2世代+新品/airpods+第2世代+新品1.csv")
- 全部のCSVを別途結合して分析しましょう。

## 注意点

1. データフレームの列名はこちらで

    - 商品名:product_name
    - 送料無料かどうか: is_free_shipping

など名前つけてるので、論文全体で統一させるように適宜好きな名前に変えること。



わからんことあったらコメント読んで、コメントの言っている意味がわからなかったら遠慮なく聞いてくださいな

In [1]:
# ①
import requests
from bs4 import BeautifulSoup
import re
import os
from time import sleep
import pandas as pd
from datetime import datetime

In [2]:
 # ②
#2019-03-12みたいな日付けを作る
def formatted_today():
    import datetime
    now = datetime.datetime.now()
    today = "{0:%Y-%m-%d}".format(now)
    return today

# 表示を見やすくするための横線を引くだけ
def print_line():
    print("---------------------------------------------------------------------------------------------------------------------------------------")

# サーバーの連続アクセスを避けるために休憩時間(second)
def sleep_(sec):
    print("過剰な連続リクエスト回避のsleep:"+str(sec)+"秒")
    sleep(sec)
    

#     b=パラメータに代入する数字リストを作る。
# b= パラメータには検索結果ページに表示する商品の開始番号がはいる
# n=パラメータには検索結果ページ一ページごとの表示数がはいる
# 例えばb=101でn=100の時、101番目から200番目の商品が表示される
# 引数には検索結果の商品数を与える。
# 例:検索結果が217件でn=100なら
#  get_product_list(total_product=217,n=100)で実行
# →[1,101,201]
def b_list(total_product,n):
    page_count = total_product//n
    page_list = []
    for i in range(0, page_count+1):
        page_list.append(i*100+1)
    return page_list


# 同時に検索する検索ワードを繋げてURL用に加工する
# "airpods", "第2世代", "新品"の３ワードを繋げようと思ったら
# get_combined_params(["airpods", "第2世代","新品"])と、リストを入れる
# →"airpods+第2世代+新品"    

def get_combined_params(params):
    return "+".join(params)


b_list(1157,100)

[1, 101, 201, 301, 401, 501, 601, 701, 801, 901, 1001, 1101]

In [3]:
#　③
# 検索結果ページを全部取得するループ
# 商品詳細ページのリンク全部(100件)を返す

# 引数 
# b: 検索結果ページの開始商品番号。 1とか101とか
# products: 検索したいワードのリスト ["airpods", "第2世代","新品"]
def get_search_result_page_single(b,products):
    # 今回は終了したオークションに検索を限定する(進行中と分けると実装がめんどいので)
    # (進行中のオークションに対応させたかったら連絡ください)
    search_mode = "closedsearch"
    url_without_param = "https://auctions.yahoo.co.jp/"+search_mode+"/"+search_mode+"?n=100"
    r = requests.get(url_without_param+"&p="+get_combined_params(products)+"&va="+get_combined_params(products)+"&b="+str(b))
    soup = BeautifulSoup(r.text,"html.parser")
    detail_link_tags = soup.select(".Product__titleLink")
    return detail_link_tags


# 商品詳細ページ100ページから１件ずつデータを取得した
# 100行のDataFrameを一つ返す
# 引数
# detail_link_tags: 100個のリンク
# b: 開始番号。1とか101。この名前でcsvを保存する
# query: 検索文字列。リストじゃなくて+で繋がったやつ。 "airpods+第2世代+新品" この名前でフォルダを作る
def create_dataframe(detail_link_tags,b,query):
    # あらかじめ各列のデータを100件ずつ収める予定の空リストを作っておく。
    # これを検索結果一ページごとに空のDataFrameに代入してDataFrameを作るのが最も高速。
    is_free_shipping_list = []
    product_name_list     = []
    start_price_list      = []
    end_price_list        = []
    is_prompt_list        = []
    start_time_list       = []
    end_time_list         = []
    description_list      = []

    for link_tag in detail_link_tags:
        link_url = link_tag.get("href")
        sleep_(0.5) #まぁ0.5秒くらい開けとけば割と平気
        print(link_url+" の取得開始")
        r = requests.get(link_url)
        soup = BeautifulSoup(r.text,"html.parser")
        
#       エラーチェック: リンクは残っているが削除されている商品が途中にある
#       削除済み商品も列数にカウントしておきたいので、全てにNoneを代入し、商品説明だけ「削除済み」の旨を代入する  
#     　削除済み商品はそもそも商品名がない
        if(not soup.select(".highlightWordSearch h1")):
            print("※欠損商品発見,スキップ")
            product_name_list.append(None)
            is_free_shipping_list.append(None)
            start_price_list.append(None)
            end_price_list.append(None)
            is_prompt_list.append(None)
            start_time_list.append(None)
            end_time_list.append(None)
            description_list.append("削除済み")
            continue
            


        #   送料無料かどうか(送料無料ダミー)
        #  送料無料のボックスを持つページには.decFreeShiping クラスがある あるなら1ないなら0
        if(soup.select(".decFreeShiping")):
            is_free_shipping_list.append(1)
        else:
            is_free_shipping_list.append(0)



        #  商品名(データフレームの列名は適宜書き換えてね)
        #  商品名は
        product_name = soup.select(".highlightWordSearch h1")[0].text
        product_name_list.append(product_name)

    #     価格については開始価格、即決価格、落札価格全部取ることにする

    # 　開始価格　<td property=auction:StartPrice> に入ってる
        start_price = soup.select("td[property='auction:StartPrice']")[0].text.strip().replace(",","").replace("円", "").replace(" ","")
        start_price = int(start_price)
        start_price_list.append(start_price)

    #   即決価格か現在価格のどちらかが存在するので、どちらも「終了価格:end_price」とする
    # 　即決ダミーを後の分析のために残しておく「即決ダミー: is_prompt」即決なら1、現在価格なら0

    #   即決価格は<p property="auction:BidOrBuyPrice" class="decTxtBuyPrice Price__value">18,290円</p>に入ってる
        if(soup.select("p.decTxtBuyPrice")):
            prompt_price = soup.select("p.decTxtBuyPrice")[0].text.strip().replace(",","").replace("円","")
            prompt_price = int(prompt_price)
            end_price_list.append(prompt_price)
            is_prompt_list.append(1)
    #   現在価格は<p property="auction:Price" class="decTxtAucPrice Price__value">18,500円</p>に入っている
        elif(soup.select("p.decTxtAucPrice")):
            current_price = soup.select("p.decTxtAucPrice")[0].text.strip().replace(",","").replace("円","")
            current_price = int(current_price)
            end_price_list.append(current_price)
            is_prompt_list.append(0)

    #   開始時間と終了時間

    # 　開始時間タグは<td property="auction:StartTime">9月 11日(水)22時 53分</td>にあるにはあるが、フォーマットが面倒臭い。
    # 　下の方に<span style="display:none" property="auction:StartTimeUT">1568210002</span>なるタグを発見。「UNIX標準時」で検索。
    #   UNIX標準時からの時刻の作成はめちゃくちゃ簡単なのでこちらを採用。

        start_time = soup.select("span[property='auction:StartTimeUT']")[0].text
        start_time = int(start_time)
        start_time =  datetime.utcfromtimestamp(start_time)
        start_time_list.append(start_time)

    #   終了時間は<span style="display:none" property="auction:EndTimeUT">1568250325</span>に入ってる。
        end_time = soup.select("span[property='auction:EndTimeUT']")[0].text
        end_time = int(end_time)
        end_time = datetime.utcfromtimestamp(end_time)
        end_time_list.append(end_time)

    #   商品説明。何の情報が欲しいのかわからないのでとりあえずそのまま本文を代入
    #   商品説明は<div id="acMdUsrPrv" class="modUsrPrv highlightWordSearch">に入っている
    # 　spanタグやらに囲まれてごちゃごちゃしているので、get_text()でタグを除去
        description = soup.select("#acMdUsrPrv")[0].get_text().replace("\n"," ").replace("\xa0", " ").strip()
        description_list.append(description)

    #   ログ
        print(link_url+" の取得完了")

    print("全リンク取得完了")
    #     DataFrame作成(データフレームの列名は適宜書き換えてね)
    df = pd.DataFrame()
    df["is_free_shipping"] = is_free_shipping_list
    df["product_name"]     = product_name_list
    df["start_price"]      = start_price_list
    df["end_price"]        = end_price_list
    df["is_prompt"]        = is_prompt_list
    df["collected_at"]       = start_time_list
    df["date"]         = end_time_list

    
    #   おまけ: 開始日時と終了日時の差の日数
    #   timedelta列：日付差データ全部(日数とか時間とか)
    #   days_diff列: 日付差だけ  (timedelta.days関数で抽出できる)
    df["timedelta"]        = df["end_time"] - df["start_time"]
    days_diff_list = []
    for delta in df["timedelta"]:
        days_diff_list.append(delta.days)
    df["left_days"]        = days_diff_list
    
#     os.makedirsはフォルダを作る。exist_okをTrueにしないと、既にあるフォルダを作るとエラー
    os.makedirs(query,exist_ok=True)
#     保存して終わり。 "101.csv"
    df.to_csv(query+"/"+query+str(b)+".csv")
    


In [4]:
#　④
product_b_list = b_list(1157,100)#[1,101,201,...,1001,1101]の、b=パラメータに入れる数値を作成。
# products = ["airpods","第2世代","新品"] #ここに検索ワードを"リストで"入れる
products = ["演歌"]

for number in product_b_list:
    print(str(number)+"商品から取得")
    # b=に1や101を代入。productsにワードのリストを代入。
    # 検索結果一ページごとのリンクタグが100個帰ってくるのでtagsに代入 
    tags = get_search_result_page_single(b=number,products=products)
          
    create_dataframe(detail_link_tags=tags, b=number,query=get_combined_params(products))
    print(str(number+99)+"商品まで取得完了")
    

1商品から取得
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/n366117733 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/n366117733 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/r216032715 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/r216032715 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/f351147123 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/f351147123 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/n329912965 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/n329912965 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/p599789901 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/p599789901 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/e375449816 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/e375449816 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/l548396823 の取得開始
https://page.aucti

https://page.auctions.yahoo.co.jp/jp/auction/c758310033 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/n368498397 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/n368498397 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/u268348524 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/u268348524 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/d385886854 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/d385886854 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/x573626744 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/x573626744 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/p711567322 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/p711567322 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yahoo.co.jp/jp/auction/e381204264 の取得開始
https://page.auctions.yahoo.co.jp/jp/auction/e381204264 の取得完了
過剰な連続リクエスト回避のsleep:0.5秒
https://page.auctions.yaho

KeyError: 'end_time'

In [ ]:
# 検証
setumei = '''
<div id="acMdUsrPrv" class="modUsrPrv highlightWordSearch">


<table width="100%" border="0" cellspacing="0" cellpadding="0"><tbody><tr><td>
<p>
</p><div><table width="100%" border="0" cellspacing="0" cellpadding="0"><tbody><tr><td><div>AirPods with Charging Case MV7N2J/A 第2世代&nbsp;本体</div><div><br><div>品名:AirPods with Charging Case</div><div><div>型名:MV7N2J/A</div></div><div><br></div></div></td></tr></tbody></table></div><span>新品未開封品。第2世代</span><div>ネット通販で8月下旬購入の明細付きです</div><div><div><br></div><div>ヤフネコ宅急便コンパクトで送料無料</div><div>在庫ありますので入金確認後すぐ発送可能です</div></div><div><br></div><div><span>●新規の方からのキャンセルが続きますので（調査中）、新規の方はご遠慮ください</span></div><div><span>※落札24時間以内にご連絡、48時間以内にお支払いできる方のみご入札ください</span></div><div><div>※落札後に質問する方や、無断キャンセルする方はご遠慮ください。よろしくお願いします</div></div><div><br></div>
<p></p>
</td></tr></tbody></table>

</div>
'''

s = BeautifulSoup(setumei,"html.parser")
s.get_text().replace("\n"," ").replace("\xa0", " ").strip()


In [80]:
# 検証
df.to_csv("一ページ目商品詳細.csv")
d = pd.read_csv("一ページ目商品詳細.csv",index_col=0)
d

,is_free_shipping,product_name,start_price,end_price,is_prompt,start_time,end_time,timedelta,days_diff
0,1,★即決・新品★AirPods (エアーポッズ/第2世代) with Charging Cas...,18300,18300,1,2019-09-13 15:34:32,2019-09-13 22:38:21,0 days 07:03:49.000000000,0
1,0,[新品未使用品]AirPods (エアーポッズ/第2世代) with Charging Ca...,17000,17500,1,2019-09-11 07:05:43,2019-09-13 17:45:27,2 days 10:39:44.000000000,2
2,1,Apple AirPods2 MV7N2J/A エアーポッズ 第2世代 ワイヤレスイヤホン ...,18000,18000,1,2019-09-11 01:18:12,2019-09-13 14:28:29,2 days 13:10:17.000000000,2
3,1,[新品未使用品]AirPods (エアーポッズ/第2世代) with Charging Ca...,1,19000,1,2019-09-10 11:54:56,2019-09-13 12:54:56,3 days 01:00:00.000000000,3
4,1,新品未開封 AirPods 第2世代 MV7N2J/A,18300,18300,1,2019-09-12 13:52:18,2019-09-12 21:09:35,0 days 07:17:17.000000000,0
5,0,☆新品 未開封品 第2世代 Apple アップル AirPods with Charging...,16000,16500,0,2019-09-09 05:56:02,2019-09-12 13:54:24,3 days 07:58:22.000000000,3
6,1,[新品未使用品]AirPods with Wireless Charging Case (エ...,1,22000,1,2019-09-09 14:42:12,2019-09-12 12:42:12,2 days 22:00:00.000000000,2
7,0,新品 未使用 エアポッズ 第2世代 2019年8月1日購入 新型 Bluetooth MV7...,17000,17500,0,2019-09-08 11:19:51,2019-09-12 11:19:51,4 days 00:00:00.000000000,4
8,1,新品・国内正規★AirPods (エアーポッズ/第2世代) with Charging Ca...,18290,18290,1,2019-09-12 03:56:00,2019-09-12 10:53:34,0 days 06:57:34.000000000,0
9,1,[新品未使用品]AirPods (エアーポッズ/第2世代) with Charging Ca...,1,19500,1,2019-09-11 14:30:10,2019-09-12 05:10:19,0 days 14:40:09.000000000,0


In [82]:
#検証 UNIX標準時から日数差をとる
start = datetime.utcfromtimestamp(1568008917)
end   = datetime.utcfromtimestamp(1568212690)

df = pd.DataFrame([1],columns=["num"])
df["start"]= start
df["end"]  = end
df["difference"] = df["end"]-df["start"]
days_diff_list = []
for timedelta in df["difference"]:
    days_diff_list.append(timedelta.days)
df["day_dif"] = days_diff_list
df



,num,start,end,difference,day_dif
0,1,2019-09-09 06:01:57,2019-09-11 14:38:10,2 days 08:36:13,2
